# KOSTRA skjema N og Q


## Henter nødvendige pakker
**For løpet Kostra0N_0Q trengs følgende pakker:**  
   - Pandas
   - Numpy
   - Selvlagde funksjoner lagret i fil funksjoner.py

In [ ]:
import pandas as pd
import numpy as np
exec(open('../funksjoner.py').read())

## Angi årgang som produseres

In [ ]:
%run ./../aargang.ipynb

## Henter nødvendige datafiler
**For løpet Kostra0N_0Q trengs følgende datafiler:**
- Import fra Driller
    - Skjema KOSTRA0N
    - Skjema KOSTRA0Q
- Lister fra KLASS
    - Kodeliste for Kostra balansekapitler
    - Kommunelisten for gjeldene årgang
    - Fylkeskommunelisten for gjeldene årgang

In [ ]:
kostra0n = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/import_driller/" + aar + "/kostra0n.parquet")
kostra0q = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/import_driller/" + aar + "/kostra0q.parquet")
kap = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kapittel.parquet").astype(str)
komm = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/komm.parquet").astype(str)
fykomm = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fykomm.parquet").astype(str)

## Bearbeider regnskapsfilene

### Slår sammen Kostra0N og Kostra0Q

In [ ]:
#Setter sammen skjema a og c
nq = pd.concat([kostra0n, kostra0q])

### Ordner data
- Definerer rett datatype
- Endrer kolonnenavn
    - Fra store til små bokstaver
    - Endrer navn
- Fjerner manglende observasjoner

In [ ]:
nq = nq.astype({'FUNKSJON_KAPITTEL':str,'ART_SEKTOR':str, 'KONTOKLASSE':str,'PERIODE':str,'REGION':str})

In [ ]:
#Endrer headingen fra store til små bokstaver
nq.columns = [x.lower() for x in nq.columns]

In [ ]:
#Endrer navn på kapittel
nq.rename(columns={'funksjon_kapittel':'kapittel'}, inplace = True)

In [ ]:
nq = nq.groupby(['kapittel', 'kontoklasse', 'periode', 'region'])[['belop']].sum().reset_index()

In [ ]:
#Filtrerer bort alle rader med manglede verdier. Fungerer det? Må testes på et annet datasett som faktisk har manglende verdier... 
nq_null = nq.dropna()

#### Kontrollerer fjernede rader

In [ ]:
kontroll_merge(nq,nq_null)

### Snur fortegn for inntektsarter
- Endrer alle verdier som har art høyere eller lik 600 til negative tall

In [ ]:
nq_snudd = nq_null.copy()

In [ ]:
nq_snudd['belop']=np.where((nq_snudd.kapittel >= '31') & (nq_snudd.kapittel <= '5990'),  nq_snudd.belop*-1, nq_snudd.belop)

## Lager Oslo Fylkeskommune
- Tallene fra Oslo skal være med både for kommune og fylkeskommune. Vi må derfor kopiere alle Oslo tall og endre region på de kopierte radene fra 0301 til 0300. 

In [ ]:
#Henter ut Oslo og endrer regionskode til 0300. Legger Oslo fylkeskommune på arbeidsfilen. 
oslo=nq_snudd[(nq_snudd['region']=='0301')].copy()
oslo['region'] = oslo['region'].str.replace('0301', '0300')

- Slår sammen Oslo Fylkeskommune med regnskapsfilen

In [ ]:
nq_forts = pd.concat([nq_snudd, oslo])

## Filtrerer gyldige kapitler
**Kapittel listen**
- Hentes direkte fra KLASS via API i egen notebook (API Klass). 
- Filen laget i API-notebooken er lastet inn helt i starten av denne boken.

**Kontroll**
- Funksjonen kontroll_merge er laget i egen notebook (funksjoner.py)
- Kontrollen lister ut alle rader som er fjernet i mergen over. Dersom ingen rader listes ut ble ingenting fjernet i operasjonen over.

### Kapittelsfilter
- Merger alle gylige kapitler med regnskapsfilen. Dette fjerner alle reder med ugyldige kapitler

In [ ]:
kap=kap.drop(columns={'periode'})

In [ ]:
gyldige_kap = list(kap.columns.values)

In [ ]:
k1 = nq_forts.set_index(gyldige_kap).index
k2 = kap.set_index(gyldige_kap).index
kapittelfilter = nq_forts[k1.isin(k2)]

#### Kontrollerer fjernede rader
- Må kunne se nærmere på dataen

In [ ]:
kontroll_merge(nq_forts, kapittelfilter)

# Foreløpig Fil - kontrollert

In [ ]:
kostra0n_0q_kontrollert = kapittelfilter

## Finner manglende kommuner
- For å finne hvem som mangler må vi koble opp mot regionslisten. Dette gjøres via API mot Klass (notebook API Klass).

- Lagt opp mot kommuner og fylkeskommuner, ikke bydeler


In [ ]:
#Lister ut alle kommuner og fylkeskommuner som er med i regnskapsfilen. 
#Lagrer filen til senere bruk.
levert = kostra0n_0q_kontrollert[['region']].drop_duplicates()
#levert.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/kostra0b_0d_levert_" + aar + ".parquet")

### Alle regioner

In [ ]:
#Slår sammen kommujne og fylkeskommunelisten tidligere hentet fra Klass. 
regioner = pd.concat([komm, fykomm])

### Manglende kommuner

In [ ]:
#Lister ut manglende kommuner
manglende = kontroll_merge(regioner,levert)
manglende

In [ ]:
#manglende.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/kostra0n_0q_manglende_" + aar + ".parquet")

## Oppretter regnskapsomfang
- Regnskapsomfang C definerer at dette er konsoliderte tall

In [ ]:
kostra0n_0q_kontrollert['regnskapsomfang'] = 'C'

# Ferdig fil
- Lagrer fil som er klar som input til KostraRegnskaps-metode
- Her må vi ha mulighet for å se på dataen

In [ ]:
kostra0n_0q_kontrollert.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/kostra0n_0q_basis_" + aar + ".parquet")

#### Kontrollfilter

In [ ]:
#sjekk=bd_basis[(bd_basis['region']=='4601') & (bd_basis['art']=='905') ].copy()
#sjekk

In [ ]:
kostra0n_0q_kontrollert